In [1]:
import pandas as pd
import numpy as np
input_data_raw = pd.read_csv('desktop/advent_of_code_day2.csv')
input_data_raw.head()

,Level 1,Level 2,Level 3,Level 4,Level 5,Level 6,Level 7,Level 8,Level 9,Level 10
0,16,17,18,21,23,24.0,27.0,24.0,NaN,NaN
1,74,76,79,81,82,85.0,85.0,NaN,NaN,NaN
2,48,51,53,54,55,59.0,NaN,NaN,NaN,NaN
3,29,31,32,34,36,39.0,41.0,46.0,NaN,NaN
4,9,12,9,11,14,16.0,17.0,20.0,NaN,NaN


In [2]:
row_differences = input_data_raw.diff(axis=1) #calculate the diffs between each column
row_differences.head()

,Level 1,Level 2,Level 3,Level 4,Level 5,Level 6,Level 7,Level 8,Level 9,Level 10
0,NaN,1,1,3,2,1.0,3.0,-3.0,NaN,NaN
1,NaN,2,3,2,1,3.0,0.0,NaN,NaN,NaN
2,NaN,3,2,1,1,4.0,NaN,NaN,NaN,NaN
3,NaN,2,1,2,2,3.0,2.0,5.0,NaN,NaN
4,NaN,3,-3,2,3,2.0,1.0,3.0,NaN,NaN


In [3]:
# now we write a function to look at each row within row_differences and check to see if it is deemed "safe"
def safe_levels(row):
    row = pd.to_numeric(row, errors='coerce') #Cast to numeric for comparison, for values that cant convert, convert to NaN
    row = row.dropna() # drop the NaNs
    
    if len(row) == 0:  #if all values were removed in the dropna() above we cannot confidently say the row is safe
        return False

    increasing = all(row > 0) #if all the values in each row in row_differences is positive, then its entirely increasing and meets part 1 of safety criteria
    decreasing = all(row < 0) #if all the values in each row in row_differences is negative, then its entirely decreasing and meets part 1 of safety criteria

    # second safety criteria is to make sure the increments are atleast 1 and at most 3. 
    for diff in row:
        if not (1 <= abs(diff) <= 3):  
            return False

    return increasing or decreasing


input_data_raw['Safe'] = row_differences.apply(safe_levels, axis=1) #create a new column that shows whether or not the row is safe


print(input_data_raw[['Safe']].value_counts())


Safe 
True     585
False    415
Name: count, dtype: int64


In [5]:
###Part 2
#My thought here is to do the same thing we did above except with a new twist that will loop through all FALSE rows, remove a single value, check if its safe with its removal and if safe, call it safe.  If we loop thru each row and its still false then we will call it false

In [13]:
def modified_safe_levels(row):
    row = pd.to_numeric(row, errors='coerce')  # put values in rows to numeric, if it fails cast to NaN
    row = row.dropna()  # punt the NaNs to the moon

    if len(row) == 0:  #if all values were removed in the dropna() above we cannot confidently say the row is safe
        return False

    diffs = row.diff().dropna()  # calculate the diffs only between consecutive values, once again punt NaNs to moon


    increasing = all(diffs > 0) #if all the values in each row in the diffs is positive, then its entirely increasing and meets part 1 of safety criteria
    decreasing = all(diffs < 0) #if all the values in each row in diffs is negative, then its entirely decreasing and meets part 1 of safety criteria

    for diff in diffs:
        if not (1 <= abs(diff) <= 3):  # second safety criteria is to make sure the increments are atleast 1 and at most 3. 
            return False

    return increasing or decreasing

input_data_raw['Safe'] = row_differences.apply(modified_safe_levels, axis=1) #create a new column that shows whether or not the row is safe


print(input_data_raw[['Safe']].value_counts())

Safe 
False    995
True       5
Name: count, dtype: int64


In [15]:
# Test out removing each value
row = pd.Series([1, 3, 2, 4, 5], index=["Level 1", "Level 2", "Level 3", "Level 4", "Level 5"])

# Test the loop
for i in range(len(input_data_raw)):
    modified_row = row.drop(row.index[i])  # Drop the i-th level
    is_safe = modified_safe_levels(modified_row)  # Check if the modified row is safe
    print(f"Modified Row (drop {input_data_raw.index[i]}): {modified_row.values}, Safe: {is_safe}")  # Display safety status

    if is_safe: 
        print("This modified row is safe.")
        break  # if its safe
    print("No modified row was deemed safe.")  

Modified Row (drop 0): [3 2 4 5], Safe: False
No modified row was deemed safe.
Modified Row (drop 1): [1 2 4 5], Safe: True
This modified row is safe.


In [14]:
def check_row_safety(row):
    """
    Check if a row is safe, either directly or by removing one column.
    """
    # Iterate through each column in the row
    for i in range(len(row)):
        # Drop the i-th column
        modified_row = row.drop(row.index[i])
        
        # Check if the modified row is safe
        is_safe = modified_safe_levels(modified_row)
        if is_safe:
            return True  # If any modified row is safe, return True
    
    # If no modified row is safe
    return False

# Apply the safety check function to each row of the DataFrame
input_data_raw['Secondary Safe'] = input_data_raw.apply(check_row_safety, axis=1)

# Count the number of rows that are deemed "safe"
safe_count = input_data_raw['Secondary Safe'].sum()

# Display the result
print(f"Number of rows deemed safe: {safe_count}")

Number of rows deemed safe: 587
